# Lecture 5 - Databasin'

## What is a database?
A database is a collection of information organised for secure storage and efficient retrieval.  They are especially useful for storing large and complex datasets.    

A flat file database stores all its information in a single table.  A relational database stores data as a collection of tables, each a distinct 'unit' of information.  This greatly increases the complexity of information a database is capable of storing, as well as the efficiency of storing it.  Tables are connected to each other using common variables, or 'keys'. When designed properly, database tables will follow the conventions of the tidyverse (from Lecture 3)

Saving data as a flat file wastes space by repeating information, and also make it difficult to update information because it exists in multiple places.  The problems are exacerbated the larger your database gets.  

The database below contains observations of sharks recorded during a series of reef surveys on different islands in the Bahamas.  Imagine updating a species name, or changing the visibility for one of the surveys.  

In [ ]:
flat_file <- read.csv('flat_file.csv')
flat_file[1:20,]

When the above flat file is translated into relational tables, it looks like this...

In [ ]:
#call all database tables and read them into the global environment
db_tables <- c("surveys.csv", "species.csv", "observations.csv", "sites.csv")                                   
list2env(lapply(setNames(db_tables, make.names(gsub("*.csv$", "", db_tables))), read.csv), envir = .GlobalEnv)
#take a look at each table
head(sites)
head(surveys)
head(observations)
head(species)

## Entity Relationship Diagrams

Databases are great for storing large and complex datasets, but they are difficult to read or see how they are organised. It's not always clear how the tables are connected or what information goes in each table. This is where database diagrams come in (the technical name for this is Entity Relationship Diagram or ERD). They are a really important tool for anybody building or querying a database. 

ERD showing relational database tables
<img src="ERD.png">


Relational databases are good for storing large and complex datasets. They are not usually 'built' in R, but rather in Python, SQL or Access BUT R is an excellence tool for querying databases. As a researcher you are more likely to spend time getting data out of databases rather than building them.  Knowing how to extract and visualise data from a database so that other humans can see what's in it is a very useful skill

#### Additional Resources about Databases
Intro to relational databases: https://www.youtube.com/watch?v=wR0jg0eQsZA  
Database management systems http://nbviewer.jupyter.org/github/fonnesbeck/Bios366/blob/master/notebooks/Section8_2-Database-Programming.ipynb
Entity Relationship Diagrams  https://www.youtube.com/watch?v=QpdhBUYk7Kk

## DPLYR

### aka the best R package ever

dplyr will allow you to manipulate data any way you please, all with a few simple functions.  

In [ ]:
#install.packages("dplyr")
library(dplyr)

### Select

The SELECT function lets you choose columns from a dataframe

In [ ]:
#select survey id and depth from the survey table
tmp <- select(surveys, id, depth)
head(tmp)

### Filter

The FILTER function lets you choose rows in a dataframe

In [ ]:
#find all the survey sights which are in Bimini (site 2)
filter(surveys, site_id==2)

#find all survey sights which are in Bimini and have a visibility value of 8
filter(surveys, site_id==2, visibility==8)

---
## Task 1
---

Filter for all observations where the number of sharks observed was greater than 1

In [ ]:
#Your code here



#### Other useful things to do in a filter function:

In [ ]:
#Search for rows that are not blank, or not NA
filter(observations, comment!="")  
filter(observations, is.na(comment)==FALSE)

#Search for a string of values  
filter(sites, name %in% c('Bimini', 'New Providence'))

#Search for rows matching multiple criteria from different columns, criteria can be additive or non-exlusive  
#e.g. find species where family is Carcharhinidae AND genus is Carcharhinus  
filter(species, family=="Carcharhinidae", genus=="Carcharhinus")   
#e.g. find surveys where depth is less than 5 OR visibility equals 12  
filter(surveys, depth<5 | visibility==12)  

### Piping   

dplyr allows you to use two types of syntax to run functions. You can use wrappping (first function goes on the inside, last one goes on the outside) OR you can use piping, which makes your functions chronological.  Piping is also called chaining.

https://www.rdocumentation.org/packages/dplyr/versions/0.4.3/topics/chain

In [ ]:
#Search for all surveys in Bimini, and select only survey number and site number
#without piping
filter(select(surveys, id, site_id), site_id==2)

#with piping
surveys %>% select(id, site_id) %>% filter(site_id==2)

---
## Task 2
---

Using piping, select species id, survey id and count from the observations table, then filter for all observations where species id is 26 

In [ ]:
#Your code here



### Arrange

The ARRANGE function lets you sort dataframes by a particular column or set of columns

In [ ]:
#arrange species by family then genus
species %>% arrange(family, genus)

#The default order is ascending. To sort descending, wrap column name with desc() function
species %>% arrange(desc(family))

### Summarise

The summarise function in dplyr is like a pivot table in excel, or like xtab (from Lecture 4).  It allows you to group data in your table and reduce multiple values down to a single value, based on a summary function like maximum value or average. 

In [ ]:
# Search for the maximum count value per species
observations %>% 
        group_by(species_id) %>%
        summarise(max_count=max(count))

In [ ]:
# Count the number of surveys in each site
surveys %>% 
        group_by(site_id)  %>%
        summarise(n_surveys=n())

---
## Task 3
---

 Use the summarise function to calculate the mean visibility per site, then arrange the table in descending site order.  
 
 Tip - use the ?summarise function or google to find how to calculate the mean.   

In [ ]:
#Your code here



### Mutate

dplyr allows you to add new columns to an existing table using the mutate function.  

things you can do with mutate: https://dplyr.tidyverse.org/reference/mutate.html

In [ ]:
#convert depth values from meters to feet
tmp4 <- surveys %>% mutate(depth_ft=depth*3.28)
head(tmp4)

#add a notes column to the species table
tmp5 <- species %>% mutate(notes="")
tmp5$notes[tmp5$common_name=="Tiger shark"] <- "This is my favourite shark"
head(tmp5)

## Joins

Join functions allow you to connect multiple tables based on common variables(columns) so that you can access info from multiple tables at once.  There are (believe it or not) 6 different ways to join tables.  The 2nd page of the R data wrangling cheat sheet has a nice summary.

The most useful of these is a **left join**, which takes a table (say table 1), and then adds all matching records from table 2.

R will automatically detect which columns two tables have in common IF they are called the same thing. If not, you have to feed the matching columns names into the join function.

In [ ]:
#add site names to the survey table
ex1 <- left_join(surveys, sites, by=c("site_id"="id")) %>% rename(site_name=name)
head(ex1)

In [ ]:
head(surveys)

### Types of Joins

**left join:** resulting table will show all records from table 1 (the left hand table), and matching records from table 2 (the right hand table)  

**right join:** result shows all records from table 2, and matching records from table 1

**inner join:** result shows only 'matched' records - where there is a matching value in both tables  

**full join:** result shows all records from both tables  

**anti join:** removes all records from table 1 that have a match in table 2   

**semi join:** removes all records from table 2 that have a match in table 1  

Anti and Semi joins are especially useful when exploring messy data

In [ ]:
#Use anti join to find unmatched records between the surveys and sites table
anti_join(surveys, sites, by=c("site_id"="id"))

#these records are returned because there is no site 6 in the sites table

In [ ]:
#Use full join to see all records from both sides, where site id=6
#since there is no site 6 in the site table, 'name' comes up as NA

full_join(surveys, sites, by=c("site_id"="id")) %>% filter(site_id==6)

Lots of tables in databases have a column called 'id' or similar in them. So when joining tables together it's useful to be able to rename columns so you remember which data came from which table, or just to avoid having lots of columns called id.

In [ ]:
ex2 <- left_join(surveys, sites, by=c("site_id"="id")) %>% rename(site_name=name)
head(ex2)

---
## Task 4
---

Join the observations and species tables so that each observation is accompanied by the family, genus, species and common name of the shark observed.  Make sure only species that *have* been observed appear in the resulting table. Then rename the 'id' column from the observations table to be 'observation id' instead.  

In [ ]:
# Your code here



---
## Task 5
---

Create a summary table that shows the total number of sharks observed per site per species.  Make sure to include the genus and species names in the table

In [ ]:
# Your code here



### Additional Resources for dplyr
intro to dplyr https://cran.r-project.org/web/packages/dplyr/vignettes/dplyr.html  
data wrangling cheatsheet https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf  

## Tidyr  (if there's time)

Tidyr is another R package that is part of the tidyverse.  It has a few functions that are useful for re-arranging datasets, either to summarise data or just to make tables easier to read.

In [ ]:
library(tidyr)

### Spread

Spread takes rows from a given column and re-organises them into columns, then fills in the corresponding records with a value of your choice (based on another existing column)

In [ ]:
#make each species name a column heading, and fill in species count as the value under each column
#first make a table with observations and species names in it
ex3 <- left_join(observations, species, by=c("species_id"="id")) %>% select(survey_id, count, common_name)
ex4 <- spread(ex3, common_name, count)
head(ex4)

### Gather

Gather takes columns and re-organises them into rows

In [ ]:
#take the spread table we just made, and transform it back so each row is a species with a count per survey
#gather(table name, name of new column, name of second new column, range of columns to transform)
gather(ex4, common_name, count, 2:14)[1:10,]

#notice that gather will not automatically filter out empty values, you have to do this yourself

# What have you learned and what's next?

The point of today's lab was to learn a few key aspects of R objects so we can wrangle them

**You should at this point be comfortable:**
 1. What is a database
 2. The best R package ever
 3. Arranging, summarizing, and mutating data
 4. How to join in various ways
 5. A couple of tidyr things

Next week we will start talking about time and how to cut it up and process it.


---
# ** A bientôt ** !